In [1]:
import pandas as pd
import numpy as np 

In [2]:
universal_report = pd.read_parquet(r'D:\python\financial statement prj\chatbot_financial_statement\csv\v3\bank_financial_report.parquet')
universal_report

,category_code,data,stock_code,quarter,year,date_added
0,IS_003,19314969.0,BID,0,2015,2015-12-30
1,IS_003,23393613.0,BID,0,2016,2016-12-30
2,IS_003,30955331.0,BID,0,2017,2017-12-30
3,IS_003,34720848.0,BID,0,2018,2018-12-30
4,IS_003,35977808.0,BID,0,2019,2019-12-30
...,...,...,...,...,...,...
120384,BS_200,425351476.0,VPB,3,2023,2023-09-30
120385,BS_200,471144692.0,VPB,4,2023,2023-12-30
120386,BS_200,491313258.0,VPB,1,2024,2024-03-30
120387,BS_200,525158092.0,VPB,2,2024,2024-06-30


In [3]:
import pandas as pd

def calculate_TTM(financial_report):

    # Remove annual data (quarter = 0)
    financial_report = financial_report[financial_report['quarter'] != 0]

    # Sort for proper calculation
    financial_report = financial_report.sort_values(by=['stock_code', 'category_code', 'year', 'quarter'])

    # Create a period column to track time
    financial_report['period'] = financial_report['year'] * 4 + financial_report['quarter'] - 1

    # Compute the rolling sum over the last 4 quarters
    financial_report['data_TTM'] = (
        financial_report.groupby(['stock_code', 'category_code'])['data']
        .transform(lambda x: x.rolling(window=4, min_periods=4).sum())
    )

    # Assign date_added from the latest quarter within the 4-quarter window
    financial_report['date_added_TTM'] = (
        financial_report.groupby(['stock_code', 'category_code'])['date_added']
        .transform(lambda x: x.shift(0))
    )

    # Keep only rows where TTM is valid (i.e., at least 4 quarters exist)
    financial_report = financial_report.dropna(subset=['data_TTM'])

    # Rename category_code to include _TTM
    financial_report['category_code'] = financial_report['category_code'] + '_TTM'

    # Select required columns
    return financial_report[['stock_code', 'category_code', 'year', 'quarter', 'data_TTM', 'date_added_TTM']].rename(
        columns={'data_TTM': 'data', 'date_added_TTM': 'date_added'}
    )
report_ttm= calculate_TTM(universal_report)

In [10]:
result = report_ttm[(report_ttm['category_code'] == 'IS_020_TTM') & 
                    (report_ttm['stock_code'] == 'TCB') & 
                    (report_ttm['year'].isin([2022, 2023]))]
print(result)

      stock_code category_code  year  quarter       data date_added
85236        TCB    IS_020_TTM  2022        1 -4968330.0 2022-03-30
85237        TCB    IS_020_TTM  2022        2 -5196102.0 2022-06-30
85238        TCB    IS_020_TTM  2022        3 -5413367.0 2022-09-30
85239        TCB    IS_020_TTM  2022        4 -5131342.0 2022-12-30
85240        TCB    IS_020_TTM  2023        1 -5046817.0 2023-03-30
85241        TCB    IS_020_TTM  2023        2 -4753497.0 2023-06-30
85242        TCB    IS_020_TTM  2023        3 -4580043.0 2023-09-30
85243        TCB    IS_020_TTM  2023        4 -4697362.0 2023-12-30


In [15]:
result_2 = universal_report[(universal_report['category_code'] == 'IS_020') & 
                    (universal_report['stock_code'] == 'TCB') & 
                    
                    (universal_report['year'].isin([2022, 2023]))]

In [16]:
print(result_2)

      category_code       data stock_code  quarter  year date_added
14722        IS_020 -5131342.0        TCB        0  2022 2022-12-30
14723        IS_020 -4697362.0        TCB        0  2023 2023-12-30
85236        IS_020 -1170786.0        TCB        1  2022 2022-03-30
85237        IS_020 -1438881.0        TCB        2  2022 2022-06-30
85238        IS_020 -1347539.0        TCB        3  2022 2022-09-30
85239        IS_020 -1174136.0        TCB        4  2022 2022-12-30
85240        IS_020 -1086261.0        TCB        1  2023 2023-03-30
85241        IS_020 -1145561.0        TCB        2  2023 2023-06-30
85242        IS_020 -1174085.0        TCB        3  2023 2023-09-30
85243        IS_020 -1291455.0        TCB        4  2023 2023-12-30


In [18]:
numbers = [ -1145561.0 , -1086261.0,-1174136.0,-1347539.0 ]
total_sum = sum(numbers)
print(total_sum)

-4753497.0


In [19]:
def process_financial_statements(financial_report):
    # Filter IS and CF accounts for TTM calculation
    is_report = financial_report[financial_report['category_code'].str.startswith('IS')]
    cf_report = financial_report[financial_report['category_code'].str.startswith('CF')]

    # Calculate TTM values
    is_report_TTM = calculate_TTM(is_report.copy())
    cf_report_TTM = calculate_TTM(cf_report.copy())

    # Combine original data with TTM data
    final_report = pd.concat([financial_report, is_report_TTM, cf_report_TTM], ignore_index=True)

    return final_report

final_report = process_financial_statements(universal_report)

In [25]:
final_report[(final_report['category_code'] == 'CF_001_TTM') & 
                    (final_report['stock_code'] == 'TCB') & 
                    
                    (final_report['year'].isin([2022, 2023]))]

,category_code,data,stock_code,quarter,year,date_added
155399,CF_001_TTM,36155336.0,TCB,1,2022,2022-03-30
155400,CF_001_TTM,37502440.0,TCB,2,2022,2022-06-30
155401,CF_001_TTM,40766412.0,TCB,3,2022,2022-09-30
155402,CF_001_TTM,43558389.0,TCB,4,2022,2022-12-30
155403,CF_001_TTM,48102369.0,TCB,1,2023,2023-03-30
155404,CF_001_TTM,50913194.0,TCB,2,2023,2023-06-30
155405,CF_001_TTM,53312172.0,TCB,3,2023,2023-09-30
155406,CF_001_TTM,53749365.0,TCB,4,2023,2023-12-30


In [ ]:
combined_report = pd.concat([universal_report, report_4NQ], ignore_index=True)
combined_report

,data,stock_code,quarter,year,date_added,category_code
0,1.931497e+07,BID,0,2015,2015-12-30,IS_150
1,2.339361e+07,BID,0,2016,2016-12-30,IS_150
2,3.095533e+07,BID,0,2017,2017-12-30,IS_150
3,3.472085e+07,BID,0,2018,2018-12-30,IS_150
4,3.597781e+07,BID,0,2019,2019-12-30,IS_150
...,...,...,...,...,...,...
1081298,5.483616e+04,YEG,3,2023,2023-09-30,CF_130_4NQ
1081299,3.213779e+04,YEG,4,2023,2023-12-30,CF_130_4NQ
1081300,2.752085e+04,YEG,1,2024,2024-03-30,CF_130_4NQ
1081301,2.727661e+04,YEG,2,2024,2024-06-30,CF_130_4NQ


: 

# add in map

In [39]:
map_universal = pd.read_csv(r'D:\python\financial statement prj\chatbot_financial_statement\csv\v3\map_category_code_universal.csv')
map_universal
                            

,Corp,corp_code,Securities,sec_code,Bank,bank_code,en_caption,category_code,parent_code
0,A. CURRENT ASSETS,BS_100,A. CURRENT ASSETS,BS_100,NaN,NaN,(Balance sheet) A. CURRENT ASSETS,BS_100,BS_100
1,NaN,NaN,I. Financial Assets,BS_110,NaN,NaN,(Balance sheet) (Securities) Financial Assets,BS_101,BS_101
2,I. Cash and cash equivalents,BS_110,1. Cash and cash equivalents,BS_111,NaN,NaN,(Balance sheet) Cash and cash equivalents,BS_110,BS_110
3,Cash,BS_111,1.1. Cash,BS_111.1,"Cash, precious metals, gemstones",BS_110,(Balance sheet) Cash,BS_111,BS_110
4,NaN,NaN,NaN,NaN,Deposits at the Central Bank,BS_120,(Balance sheet) (Bank) Deposits at the Central...,BS_112,BS_110
...,...,...,...,...,...,...,...,...,...
340,Lưu chuyển tiền thuần từ hoạt động tài chính,CF_040,Lưu chuyển tiền tệ từ hoạt động tài chính,CF_080,III.Lưu chuyển tiền thuần từ hoạt động tài chính,CF_041,(Cash flow) Net cash flow from financing activ...,CF_090,CF_090
341,Lưu chuyển tiền thuần trong kỳ (50 = 20+30+40),CF_050,LƯU CHUYỂN TIỀN THUẦN TRONG KỲ,CF_090,IV.Lưu chuyển tiền thuần trong kỳ,CF_042,(Cash flow) Net cash flow during the period,CF_100,CF_100
342,Tiền và tương đương tiền đầu kỳ,CF_060,TIỀN VÀ TƯƠNG ĐƯƠNG TIỀN ĐẦU KỲ,CF_101,V. Tiền và các khoản tương đương tiền tại thời...,CF_043,(Cash flow) Cash and cash equivalents at the b...,CF_110,CF_110
343,Ảnh hưởng của thay đổi tỷ giá hối đoái quy đổi...,CF_061,NaN,NaN,VI. Điều chỉnh ảnh hưởng của thay đổi tỷ giá,CF_044,(Cash flow) Effects of exchange rate changes o...,CF_120,CF_120


In [43]:
def add_4NQ_records(map_universal):
 
    filtered_df = map_universal[map_universal['category_code'].str.startswith(('IS_', 'CF_'), na=False)].copy()

    filtered_df['category_code'] = filtered_df['category_code'] + '_4NQ'
    filtered_df['corp_code'] = filtered_df['corp_code'].astype(str) + '_4NQ'
    filtered_df['sec_code'] = filtered_df['sec_code'].astype(str) + '_4NQ'
    filtered_df['bank_code'] = filtered_df['bank_code'].astype(str) + '_4NQ'

    # Append ' (4 nearest quarters)' suffix to relevant name columns (handling NaN values)
    filtered_df['en_caption'] = filtered_df['en_caption'].astype(str) + ' (4 nearest quarters)'
    filtered_df['Corp'] = filtered_df['Corp'].astype(str) + ' (4 nearest quarters)'
    filtered_df['Securities'] = filtered_df['Securities'].astype(str) + ' (4 nearest quarters)'
    filtered_df['Bank'] = filtered_df['Bank'].astype(str) + ' (4 nearest quarters)'

    # Handle NaN cases (if some columns had missing values)
    filtered_df.replace({'nan (4 nearest quarters)': None}, inplace=True)
    filtered_df.replace({'nan_4NQ': None}, inplace=True)

    # Append the new records to the original table
    map_universal_extended = pd.concat([map_universal, filtered_df], ignore_index=True)

    return map_universal_extended

# Apply the transformation
map_universal_extended = add_4NQ_records(map_universal)
map_universal_extended

,Corp,corp_code,Securities,sec_code,Bank,bank_code,en_caption,category_code,parent_code
0,A. CURRENT ASSETS,BS_100,A. CURRENT ASSETS,BS_100,NaN,NaN,(Balance sheet) A. CURRENT ASSETS,BS_100,BS_100
1,NaN,NaN,I. Financial Assets,BS_110,NaN,NaN,(Balance sheet) (Securities) Financial Assets,BS_101,BS_101
2,I. Cash and cash equivalents,BS_110,1. Cash and cash equivalents,BS_111,NaN,NaN,(Balance sheet) Cash and cash equivalents,BS_110,BS_110
3,Cash,BS_111,1.1. Cash,BS_111.1,"Cash, precious metals, gemstones",BS_110,(Balance sheet) Cash,BS_111,BS_110
4,NaN,NaN,NaN,NaN,Deposits at the Central Bank,BS_120,(Balance sheet) (Bank) Deposits at the Central...,BS_112,BS_110
...,...,...,...,...,...,...,...,...,...
496,Lưu chuyển tiền thuần từ hoạt động tài chính (...,CF_040_4NQ,Lưu chuyển tiền tệ từ hoạt động tài chính (4 n...,CF_080_4NQ,III.Lưu chuyển tiền thuần từ hoạt động tài chí...,CF_041_4NQ,(Cash flow) Net cash flow from financing activ...,CF_090_4NQ,CF_090
497,Lưu chuyển tiền thuần trong kỳ (50 = 20+30+40)...,CF_050_4NQ,LƯU CHUYỂN TIỀN THUẦN TRONG KỲ (4 nearest quar...,CF_090_4NQ,IV.Lưu chuyển tiền thuần trong kỳ (4 nearest q...,CF_042_4NQ,(Cash flow) Net cash flow during the period (4...,CF_100_4NQ,CF_100
498,Tiền và tương đương tiền đầu kỳ (4 nearest qua...,CF_060_4NQ,TIỀN VÀ TƯƠNG ĐƯƠNG TIỀN ĐẦU KỲ (4 nearest qua...,CF_101_4NQ,V. Tiền và các khoản tương đương tiền tại thời...,CF_043_4NQ,(Cash flow) Cash and cash equivalents at the b...,CF_110_4NQ,CF_110
499,Ảnh hưởng của thay đổi tỷ giá hối đoái quy đổi...,CF_061_4NQ,None,None,VI. Điều chỉnh ảnh hưởng của thay đổi tỷ giá (...,CF_044_4NQ,(Cash flow) Effects of exchange rate changes o...,CF_120_4NQ,CF_120


In [ ]:
def process_financial_statements(input_parquet_path: str, output_parquet_path: str,company_type:str = None) -> pd.DataFrame: 
    """
    Processes financial statements and computes TTM rolling sums.
    """
    combined_report = None
    if os.path.exists(input_parquet_path):

        financial_report = pd.read_parquet(input_parquet_path)

        financial_report = financial_report.dropna(subset=['category_code'])

        def calculate_TTM(df):
            df_sorted = df.sort_values(by=['stock_code', 'category_code', 'year', 'quarter']).reset_index(drop=True)
            df_sorted = df_sorted[df_sorted['quarter'] != 0]
            df_sorted['period'] = df_sorted['year'] * 4 + df_sorted['quarter'] - 1  

            df_lookup = df_sorted[['stock_code', 'category_code', 'year', 'quarter', 'data', 'period']].copy() # dataframe : stock_code, category_code, year, quarter, data, period, lookback1, lookback2, lookback3, lookback4
            df_lookup['lookback1'] = df_lookup['period']
            df_lookup['lookback2'] = df_lookup['period'] - 1
            df_lookup['lookback3'] = df_lookup['period'] - 2
            df_lookup['lookback4'] = df_lookup['period'] - 3
            print(df_lookup)

            df_long = df_lookup.melt(
                id_vars=['stock_code', 'category_code', 'data', 'period'], #stockcode, category_code, data, period, lookback_type,lookup_period
                value_vars=['lookback1', 'lookback2', 'lookback3', 'lookback4'],
                var_name='lookback_type',
                value_name='lookup_period'
            )
            print(df_long)

            df_merged = df_long.merge(
                df_sorted[['stock_code', 'category_code', 'period', 'data']],
                left_on=['stock_code', 'category_code', 'lookup_period'],
                right_on=['stock_code', 'category_code', 'period'],
                how='left',
                suffixes=('', '_past')
            )
            print(df_merged)

            df_final = df_merged.groupby(['stock_code', 'category_code', 'period'])['data_past'].sum().reset_index()
            df_final['year'] = df_final['period'] // 4
            df_final['quarter'] = df_final['period'] % 4 + 1
            df_final.rename(columns={'data_past': 'data'}, inplace=True)

            return df_final

        print(f"Processing {company_type or 'Universal'} Financial Report")

        is_report = financial_report[financial_report['category_code'].str.startswith('IS')]
        cf_report = financial_report[financial_report['category_code'].str.startswith('CF')]

        is_report_TTM = calculate_TTM(is_report.copy()) if not is_report.empty else None
        cf_report_TTM = calculate_TTM(cf_report.copy()) if not cf_report.empty else None

        report_TTM = pd.concat([df for df in [is_report_TTM, cf_report_TTM] if df is not None], ignore_index=True)

        report_TTM['category_code'] = report_TTM['category_code'] + '_TTM'
        report_TTM['original_category_code'] = report_TTM['category_code'].str.replace('_TTM', '', regex=True)

        report_TTM = report_TTM.merge(
            financial_report[['stock_code', 'year', 'quarter', 'category_code', 'date_added']],
            left_on=['stock_code', 'year', 'quarter', 'original_category_code'],
            right_on=['stock_code', 'year', 'quarter', 'category_code'],
            how='left',
            suffixes=('', '_original')
        )
        report_TTM.drop(columns=['category_code_original', 'original_category_code'], inplace=True)

        missing_cols = [col for col in financial_report.columns if col not in report_TTM.columns]
        for col in missing_cols:
            report_TTM[col] = None

        report_TTM = report_TTM[financial_report.columns]

        combined_report = pd.concat([financial_report, report_TTM], ignore_index=True)

        combined_report.to_parquet(output_parquet_path, index=False)
        print(f"Financial statement saved to {output_parquet_path}")

    else:
        print(f"File not found: {input_parquet_path}. Skipping.")

    return combined_report